In [1]:
DBPATH = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/test_DB/"
DBNAME = "test.db"
DBRESDIR = "RMSResources/"
import sys
import logging
logger = logging.getLogger("rmspool")
logger.setLevel("INFO")
logger.addHandler(logging.StreamHandler(sys.stdout))
from rmsp import rmscore
rms = rmscore.ResourceManagementSystem(f"{DBPATH}{DBNAME}", f"{DBPATH}{DBRESDIR}")
from rmsp import rmsbuilder
rmspool = rmsbuilder.RMSProcessWrapPool(rms, 8)
rmsb = rmsbuilder.RMSUnrunTasksBuilder(rmspool)
from rmsp import rmsutils
rms.set_scriptID("93b9b3d5f2934e639886ea387b0c4f50")

Checking tasks to skip...
Checking pending tasks to submit...
Total pending pids available to submit: 0 / 0
Checking tasks to skip...
Checking pending tasks to submit...
Total pending pids available to submit: 221 / 350
WARNING! An unruntask cannot be started with exceptions! pid: a8c4eca616be4311b32c093261fe17e3
Exception('Cannot overwrite files: /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/TrimmedData/C1a_1.fq.gz;/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/TrimmedData/C1a_2.fq.gz;/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/RawData/fastp_reports/C1a.html;/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/RawData/fastp_reports/C1a.json')
Cannot overwrite files: /fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/TrimmedData/C1a_1.fq.gz;/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/TrimmedData/C1a_2.fq.gz;/fs/cbsuhy02/storage/yc2553/yc2553/projects/TR

In [2]:
from commonhelper import load_func
from commonhelper import convert_to_bool as ctb
import pandas as pd
from mphelper import ProcessWrapPool
import subprocess
import tempfile
import os

In [3]:
conda_env = "DI"
pipeline = "./procapanalysis.py"
RMSTemplate_preprocessing_with_UMI_20240501 = load_func(pipeline, "RMSTemplate_preprocessing_with_UMI_20240501")
RMSTemplate_merge_reseqs_20240501 = load_func(pipeline, "RMSTemplate_merge_reseqs_20240501")
RMSTemplate_merge_replicates_20240501 = load_func(pipeline, "RMSTemplate_merge_replicates_20240501")
RMSTemplate_peak_calling_pints_20240501 = load_func(pipeline, "RMSTemplate_peak_calling_pints_20240501")
RMSTemplate_generate_PROcap_gbratio_20240501 = load_func(pipeline, "RMSTemplate_generate_PROcap_gbratio_20240501")
RMSTemplate_generate_replicates_correlation_20240501 = load_func(pipeline, "RMSTemplate_generate_replicates_correlation_20240501")
RMSTemplate_generate_PROcap_RNA_length_20240501 = load_func(pipeline, "RMSTemplate_generate_PROcap_RNA_length_20240501")

In [4]:
PROJECT_DIR_d = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/PROcap/"
PROJECT_DIR_r = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/resources/"
PROJECT_DIR_o = "/fs/cbsuhy02/storage/yc2553/yc2553/projects/TRE_directionality/output/"

# PRO-cap preprocessing

In [5]:
chrom_size_files = f"{PROJECT_DIR_r}genomes/human/hg38.chrom.sizes.filtered"
star_nospikein_ref_dirs = f"{PROJECT_DIR_r}STAR_genome_index/hg38_rDNA/"
star_spikein_ref_dirs = f"{PROJECT_DIR_r}STAR_genome_index/hg38_rDNA_dm6/"

In [6]:
procap_info = pd.read_excel(f"{PROJECT_DIR_o}tables/SuppTable1.xlsx")
lib_names = list(procap_info["Library Name"])

## Preprocessing

In [7]:
files = []
for _, r in procap_info.iterrows():
	rna_strand = "reverse"
	lib_name = r["Library Name"]
	r1_name = r["Sequencing Read 1"]
	r2_name = r["Sequencing Read 2"]

	if type(r1_name) != str:
		continue
	
	if r["3' adapter"] == "VRA3 + 6N" and r["5' adapter"] == "VRA5 + 6N":
		adapter_seq1 = "TGGAATTCTCGGGTGCCAAGG"
		adapter_seq2 = "GATCGTCGGACTGTAGAACTCTGAAC"
		umi_loc = 'per_read'
		umi_len = 6
	elif r["3' adapter"] == "VRA3 + 6N" and r["5' adapter"] == "VRA5":
		adapter_seq1 = "TGGAATTCTCGGGTGCCAAGG"
		adapter_seq2 = "GATCGTCGGACTGTAGAACTCTGAAC"
		umi_loc = 'read1'
		umi_len = 6
	else:
		print("ERROR:", r["3' adapter"], r["5' adapter"], lib_name)
		continue
	try:
		if ctb(r["S2 spike-in"].strip()):
			star_genome_dir = star_spikein_ref_dirs
		else:
			star_genome_dir = star_nospikein_ref_dirs
	except:
		print("ERROR:", r["S2 spike-in"], lib_name)
		continue
	try:
		rms.file_from_path(f"{PROJECT_DIR_d}RawData/{r1_name}")
		rms.file_from_path(f"{PROJECT_DIR_d}RawData/{r2_name}")
	except:
		print("ERROR:", "Missing files", lib_name)
		continue
	RMSTemplate_preprocessing_with_UMI_20240501(
		rmsb,
		**{
			"conda_env": conda_env,
			"project_dir": f"{PROJECT_DIR_d}",
			"prefix": f"{lib_name}",
			"genome_dir": star_genome_dir,
			"chrom_size_file": chrom_size_files,
			"r1": f"{PROJECT_DIR_d}RawData/{r1_name}",
			"r2": f"{PROJECT_DIR_d}RawData/{r2_name}",
			"rna_strand": rna_strand,
			"umi_len": umi_len,
			"umi_loc": umi_loc,
			"adapter_seq_r1": adapter_seq1,
			"adapter_seq_r2": adapter_seq2,
			"thread": 32,
		    "remove_spliced_reads":True
		  }
	)

In [8]:
samples = ["C1", "CTCF_U", "CTCF_T", "ZNF143_U", "ZNF143_T"]

In [9]:
sample_to_replicates = {}
for sample in samples:
	if sample == "C1":
		sample_to_replicates[sample] = [f"{sample}a", f"{sample}b"]
	else:
		sample_to_replicates[sample] = [f"{sample}1", f"{sample}2"]

In [10]:
for sample, namedreplicates in sample_to_replicates.items():
	RMSTemplate_merge_replicates_20240501(
		rmsb,
		**{
			"conda_env": conda_env,
			"project_dir": PROJECT_DIR_d,
			"prefixes": namedreplicates,
			"chrom_size_file": chrom_size_files
		  }
	)

In [11]:
sample_to_namedsample = {}
for sample in samples:
	rep1, rep2 = sample_to_replicates[sample]
	sample_to_namedsample[sample] = f"brm_{rep1}_and_{rep2}_erm"

In [12]:
for sample, namedsample in sample_to_namedsample.items():
	RMSTemplate_peak_calling_pints_20240501(
		rmsb,
		**{
			"conda_env": conda_env,
			"project_dir": f"{PROJECT_DIR_d}",
			"prefix": namedsample,
			"exp_type":"PROcap", 
			"thread": 32
		  }
	)

## QC

### Gene body ratio

In [13]:
for lib_name in lib_names:
	RMSTemplate_generate_PROcap_gbratio_20240501(
		rmsb,
		**{
			"conda_env": conda_env,
			"project_dir": PROJECT_DIR_d,
			"gbratio_annotation_file": f"{PROJECT_DIR_d}QC/Annotations/gencode.v37.annotation_proteincoding_nonoverlapping_long_transcripts.gtf.gz",
			"prefix": lib_name
		  }
	)

### Median RNA length

In [14]:
for lib_name in lib_names:
	RMSTemplate_generate_PROcap_RNA_length_20240501(
		rmsb,
		conda_env = conda_env,
		project_dir = PROJECT_DIR_d,
		prefix = lib_name
	)

### Replicate correlation

In [15]:
%env MPLBACKEND=

env: MPLBACKEND=


In [16]:
for s, replicates in sample_to_replicates.items():
	RMSTemplate_generate_replicates_correlation_20240501(
		rmsb,
		conda_env = conda_env,
		project_dir = f"{PROJECT_DIR_d}",
		prefixes = replicates,
		bin_size = 50,
		chrom_size_file = f"{PROJECT_DIR_r}genomes/human/hg38.chrom.sizes.filtered.no.chrY"
	)

In [17]:
# Execute all above commands

rmspool.nthread = 16
u = rmsb.execute_builder()

In [3]:
rmspool.print_stat()

# rpm normalization

In [33]:
ks = ["pl", "mn"]

In [34]:
# For analysis in Fig.4 and related figures

bws = {}
for sample in samples:
	if sample == "C1":
		continue
	# replicates: browser shots
	for n in [1,2]:
		bws[f"{sample}{n}"] = [f"{PROJECT_DIR_d}Alignments/{sample}{n}_5{k}.bw" for k in ks]
	# merged: metaplots
	bws[sample] = [f"{PROJECT_DIR_d}Alignments/brm_{sample}1_and_{sample}2_erm_5{k}.bw" for k in ks]

In [46]:
# CTCF - HCT116: a male with colorectal carcinoma (https://www.cellosaurus.org/CVCL_0291)
# ZNF143 - HEK293T: human embryonic kidney cells from a female fetus (https://www.cellosaurus.org/CVCL_0063)

chroms = {}
for m in bws:
	if "ZNF143" in m:
		chroms[m] = ["chr" + str(n) for n in range(1,23)] + ["chrX"]
	else:
		chroms[m] = ["chr" + str(n) for n in range(1,23)] + ["chrX", "chrY"]

In [36]:
def bw_rpm(bw_ins, bw_outs, chroms):
	with tempfile.TemporaryDirectory() as tempdir:
		bws_temp = [os.path.join(tempdir, f"filtered_{i}.bw") for i in range(len(bw_outs))]
		
		# Filter BigWig files by chromosomes
		for bw_in, bw_temp in zip(bw_ins, bws_temp):
			commands = ["biodatatools filter_bigwig_by_chroms",
						"-i", bw_in,
						"-o", bw_temp,
						"-chroms", " ".join(chroms)
						]
			subprocess.run(" ".join(commands), shell=True)
		
		# Normalize BigWig files to RPM
		commands = ["biodatatools normalize_bigwig",
					"-ibws", " ".join(bws_temp),
					"-obws", " ".join(bw_outs),
					"-mode rpm"
					]
		subprocess.run(" ".join(commands), shell=True)

In [50]:
pwpool = ProcessWrapPool(len(bws))

for m in bws:
	bw_ins = []
	bw_outs = []
	for n in range(2):
		bw_in = bws[m][n]
		bw_ins.append(bw_in)
		bw_out = f"{PROJECT_DIR_d}Alignments/{m}_5{ks[n]}_rpm.bw"
		bw_outs.append(bw_out)
	if not os.path.exists(bw_out):
		pwpool.run(bw_rpm, args=[bw_ins, bw_outs, chroms[m]])

In [4]:
len(pwpool.finished_tasks)

In [54]:
pwpool.close()